The **Part 1** of the project is divided into three sections:

1- Feature Extraction (Using SIFT)

2- Outlier Removal (Using RANSAC)

3- Computing the Homographies (Using DLT)


**pip install opencv-python**

**pip install opencv-contrib-python**

**Section 1**

In [1]:
#Imports
import numpy as np
import cv2
import os

In [1]:
#Display the video
capture = cv2.VideoCapture(os.path.abspath('trymefirst_lisbon.mp4'))
while(cap.isOpened()):
    ret, frame = cap.read()
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  ##press q if you want the video to stop 
        break

cap.release()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined

In [5]:
capture = cv2.VideoCapture(os.path.abspath('trymefirst_lisbon.mp4'))
frameNr = 0
sift = cv2.SIFT_create()
while True:
    success, frame = capture.read()
    if success:
        gray= cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        kp, des = sift.detectAndCompute(gray,None)
        break



(<KeyPoint 0000018DBBF9D510>, <KeyPoint 0000018DBBFAB0C0>, <KeyPoint 0000018DABF6CF60>, <KeyPoint 0000018DABF6CBA0>, <KeyPoint 0000018DABF6C510>, <KeyPoint 0000018DABF6CC90>, <KeyPoint 0000018DABF6CF90>, <KeyPoint 0000018DABF6C5A0>, <KeyPoint 0000018DABF6C4B0>, <KeyPoint 0000018DABE7F780>, <KeyPoint 0000018DABE7F7B0>, <KeyPoint 0000018DBC2C52A0>, <KeyPoint 0000018DBC2C52D0>, <KeyPoint 0000018DBC2C5330>, <KeyPoint 0000018DBC2C5360>, <KeyPoint 0000018DBC2C5390>, <KeyPoint 0000018DBC2C53C0>, <KeyPoint 0000018DBC2C53F0>, <KeyPoint 0000018DBC2C5420>, <KeyPoint 0000018DBC2C5450>, <KeyPoint 0000018DBC2C5480>, <KeyPoint 0000018DBC2C54B0>, <KeyPoint 0000018DBC2C5540>, <KeyPoint 0000018DBC2C5510>, <KeyPoint 0000018DBC2C54E0>, <KeyPoint 0000018DBC2C5210>, <KeyPoint 0000018DBC2C51E0>, <KeyPoint 0000018DBC2C51B0>, <KeyPoint 0000018DBC2C5180>, <KeyPoint 0000018DBC2C5030>, <KeyPoint 0000018DBC2C5150>, <KeyPoint 0000018DBC2C5120>, <KeyPoint 0000018DBC2C50F0>, <KeyPoint 0000018DBC2C50C0>, <KeyPoint 000

In [15]:
# how to obtain the keypoints values from the descriptors
x = kp[1].pt[0]


3.7791266441345215


In [3]:
sift = cv2.SIFT_create()